In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../datatools')
from maneger import DataManager
from preproc import Preprocessor
# from utterance.feature import Feature
from feature import Feature

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
import spacy
nlp = spacy.load('ja_ginza')
emb_dim = nlp("形態素").vector.shape[0]
emb_dim

300

In [4]:
def load_convs(path, filename):
    with open(path+filename, "r") as f:
        json_data = json.load(f)
        # length = json_data["length"]
        convs = json_data["convs"]
    return convs

In [16]:
pre = Preprocessor()
F_path = "../X_y_data/response/"
F_name = "typeClassify_F2.dill"
featureM = DataManager(F_path, format_="dill")

Fe = featureM.load_data(F_name)
Fe.set_preprocessor(pre)

300
success load : ../X_y_data/response/typeClassify_F2.dill


In [17]:
model_path = "../models/response/"
# model_name = "typeClassify_M.pickle"
model_name = "typeClassify_SVC.pickle"
modelM = DataManager(model_path)
clf = modelM.load_data(model_name)

success load : ../models/response/typeClassify_SVC.pickle


In [18]:
cont_dir = "continue/"
cont_path = "../../corpus/nucc/" + cont_dir
length = 2
filename = "cont{0}.json".format(length)

In [19]:
convs = load_convs(cont_path, filename)

In [20]:
X_str = [conv for data in convs for conv in data]

In [21]:
X_str[:5]

[['こっからー、４０分で着けると思うけど。', 'ほんと。そうなんだ。'],
 ['ほんと。そうなんだ。', '着けんかったら、やばい。'],
 ['社会科の人。わかんない。', '社会科だね。たぶん。でもねー。'],
 ['社会科だね。たぶん。でもねー。', 'ほんとにやるのー？かなり流れてほしかったんだけど。'],
 ['だって土日でしょう、しかも。悲しーい。バイトに行きたいなあ。', '強制じゃないって。でも、それって半強制だよね。']]

In [22]:
label_list = "YN WH please proposal plain".split()
label_dict = dict( zip(label_list, range(len(label_list))) )

In [23]:
def test_X(X_law:list, mode:str):

    # X_f = Fe.featurization(X_u1)
    # y_pred = LR.predict(X_f)
    X_new = []
    # for X, y in zip(X_law, y_pred):
    #     if y == label_dict[mode]:
    #         X_new.append(X)
    for X in X_law:
        f = Fe.featurization(X[0])
        y = clf.predict(f.reshape(1,-1))
        if y == label_dict[mode]:
            X_new.append(X)
    return X_new

In [24]:
X_u1 = [ conv[0] for conv in X_str]

In [25]:
X_u1[:5]

['こっからー、４０分で着けると思うけど。',
 'ほんと。そうなんだ。',
 '社会科の人。わかんない。',
 '社会科だね。たぶん。でもねー。',
 'だって土日でしょう、しかも。悲しーい。バイトに行きたいなあ。']

In [26]:
X = test_X(X_str, mode="proposal")

In [31]:
len(X)

231

In [28]:
X2 = test_X(X_str, mode="please")

In [32]:
len(X2)

35

In [33]:
X_path = "../X_y_data/response/"
X_name = "nucc_Featulized.pickle"
XM = DataManager(X_path)

In [34]:
X_f = [Fe.featurization(X_[0]) for X_ in X_str]

In [35]:
XM.save_data(X_name, X_f)

success save : ../X_y_data/response/nucc_Featulized.pickle


In [43]:
def test_X2(X_law:list, Fs, mode:str):

    # X_f = Fe.featurization(X_u1)
    # y_pred = LR.predict(X_f)
    X_new = []
    # for X, y in zip(X_law, y_pred):
    #     if y == label_dict[mode]:
    #         X_new.append(X)
    for f, X in zip(Fs, X_law):
        # print(f)
        y = clf.predict(f.reshape(1,-1))
        if y == label_dict[mode]:
            X_new.append(X)
    return X_new

In [44]:
X2_ = test_X2(X_str, X_f, mode="YN")